In [1]:
import torch
from model import CNNNetwork
from cnn_dataset import MyDataset
from transforms import *
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import cv2

In [2]:
custom_transforms = transforms.Compose([RandomHorizontalFlip(),
                                       RandomVerticalFlip(),
                                       RandomHorizontalVerticalFlip(),
                                       Resize(),
                                       ToTensor()])

dataset = MyDataset(transform=custom_transforms)
model = CNNNetwork()
loss_fn = nn.CrossEntropyLoss()
epoch = 100

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")    
print(device)

cuda


In [4]:
model = model.to(device)
model.train()
print("Model Loaded")

Model Loaded


In [5]:
def calculate_accuracy(pred, label):
    return torch.sum(pred == label)

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
for i in range(epoch):
    data = DataLoader(dataset, batch_size=32, shuffle=True)
    correctly_predicted = 0
    total_image_no = 0
    total_loss = 0
    
    for batch_no, batch_data in enumerate(data):
        image_input = batch_data['input_image'].to(device)
        label = batch_data['label'].to(device)
        label = torch.squeeze(label, axis=1)
        
        pred = model(image_input)
        #print(pred, label)
        loss = loss_fn(pred, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss
        correctly_predicted += calculate_accuracy(label, pred)
        total_image_no += image_input.shape[0]
        
    print("Epoch:", i, "Epoch_Loss:", total_loss / total_image_no, "Accuracy", correctly_predicted / total_image_no)
        

RuntimeError: The size of tensor a (32) must match the size of tensor b (4) at non-singleton dimension 1